# Langchain <--> Elastic Search

Elasticsearch is an open source distributed, RESTful search and analytics engine, scalable data store, and vector database capable of addressing a growing number of use cases. As the heart of the Elastic Stack, it centrally stores your data for lightning-fast search, fine‑tuned relevancy, and powerful analytics that scale with ease.
Elasticsearch can store and index a variety of data, including structured and unstructured text, numerical data, and geospatial data. It's known for its ability to find queries in large-scale unstructured data
Elasticsearch uses a search index, which is similar to an index in the back of a book, to map content to its location in a document. This allows users to quickly find information without scanning through an entire document

- https://www.elastic.co/search-labs/blog/langchain-collaboration
- https://www.elastic.co/guide/en/elasticsearch/reference/current/docker.html
- https://python.langchain.com/docs/integrations/vectorstores/elasticsearch/
- https://www.elastic.co/blog/elasticsearch-is-open-source-again
- https://www.elastic.co/search-labs/blog/category/generative-ai


In [ ]:
! pip install -r requirements.txt -q

# Install ELastic Search Docker

- docker network create elastic
- docker pull docker.elastic.co/elasticsearch/elasticsearch:8.15.3
- docker run --name es01 --net elastic -p 9200:9200 -it -m 1GB docker.elastic.co/elasticsearch/elasticsearch:8.15.3

In [ ]:
import os
from dotenv import dotenv_values

In [ ]:
config = dotenv_values("./keys/.env")

In [ ]:
import os, tempfile
from langchain.prompts import PromptTemplate


from langchain_community.document_loaders import TextLoader

from langchain.chains import ConversationalRetrievalChain, RetrievalQA

from langchain_text_splitters import CharacterTextSplitter
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from google.oauth2 import service_account
from dotenv import dotenv_values
import json
import vertexai
 
import itertools
import time


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage

In [ ]:
from elasticsearch import Elasticsearch, helpers



In [ ]:
try:
    es_endpoint ="http://127.0.0.1:9200"
    es_client = Elasticsearch(
        es_endpoint,
        #api_key=os.environ.get("ELASTIC_API_KEY")
    )
except Exception as e:
    print("No Client")
    es_client=None

In [ ]:
index = "langchain-demo"

In [ ]:
prompt='''

The document schema for the profiles is as follows:

{
  "nric": "string",
  "name": "string",
  "race": "string",
  "gender": "string",
  "date_of_birth": "date",
  "age": "integer",
  "country_of_birth": "string",
  "citizenship": "string",
  "religion": "string" ["Buddhism", "Christianity", "Islam", "Hinduism", "Taoism", "No Religion"],
  "marital_status": "string" ["Single", "Married", "Divorced", "Separated", "Widowed", "Civil Partnership", "Domestic Partnership", "Engaged", "Annulled"],
  "address": {
    "block": "string",
    "street_no": "string",
    "street": "string",
    "unit": "string",
    "town": "string",
    "postal_code": "string"
  },
  "phone_number": "string",
  "email": "string",
  "occupation": "string",
  "cpf_number": "string",
  "education": {
    "highest_qualification": "string",
    "institution": "string"
  },
  "languages": {
    "spoken": {"language":"fluency" ["Basic", "Conversational", "Fluent", "Native"]},
    "written": {"language":"fluency" ["Basic", "Conversational", "Fluent", "Native"]},
  },
  "height_cm": "integer",
  "weight_kg": "integer",
  "blood_type": "string" ["A+", "A-", "B+", "B-", "O+", "O-", "AB+", "AB-"],
  "passport_number": "string",
  "drivers_license_number": "string",
  "national_service": {
    "status": "string",
    "rank": "string"
  },
  "immigration_status": "string",
  "emergency_contact": {
    "name": "string",
    "relationship": "string",
    "phone_number": "string"
  },
  "deceased": "boolean",
  "date_of_death": "date"
}

Example query:
User: Find all male Singapore citizens between 25 and 30 years old who work as software developers and speak fluent English.

Your response should be:

{
  "query": {
    "bool": {
      "should": [
        { "match": { "gender": "Male" } },
        { "match": { "citizenship": "Singapore Citizen" } },
        { "range": { "age": { "gte": 25, "lte": 30 } } },
        { "match": { "occupation": "Software Developer" } },
        {
          "match": {
            "languages.spoken.English": {
              "query": "Fluent",
              "fuzziness": "AUTO"
            }
          }
        }
      ],
      "minimum_should_match": 2
    }
  }
}

Example query 2:
User : Women who are not alive currently, who are universal blood donors born in singapore

Your response should be:

{
 "query":   {
  "bool": 
  {
     "minimum_should_match": 4,
    "should": [
            {"match": {"gender": "female"}},
            {"match": {"deceased": True}},
            {"match": {"blood_type": "o-"}},
            {"match": {"country_of_birth": "singapore"}}
            ]
   }
  }
}


Consider using multi_match for fields that might contain the value in different subfields:
{
  "multi_match": {
    "query": "Software Developer",
    "fields": ["occupation", "job_title", "role"],
    "type": "best_fields",
    "fuzziness": "AUTO"
  }
}

For names or other fields where word order matters, you might want to use match_phrase with slop:
{
  "match_phrase": {
    "full_name": {
      "query": "John Doe",
      "slop": 1
    }
  }
}

When dealing with queries that involve categories, groups, or regions (such as language families, geographical areas, or professional fields), expand the search to include all relevant specific instances. 
For example, if asked about Slavic languages, include searches for Russian, Polish, Czech, etc. If asked about people from Europe, include searches for various European countries.


Generate a JSON query for Elasticsearch. Provide only the raw JSON without any surrounding tags or markdown formatting, because we need to convert your response to an object. 
Use a lenient approach with 'should' clauses instead of strict 'must' clauses. Include a 'minimum_should_match' parameter to ensure some relevance while allowing flexibility. Avoid using 'must' clauses entirely.
All queries must be lowercase.

Use 'match' queries instead of 'term' queries to allow for partial matches and spelling variations. Where appropriate, include fuzziness parameters to further increase tolerance for spelling differences. 
For name fields or other phrases where word order matters, consider using 'match_phrase' with a slop parameter. Use 'multi_match' for fields that might contain the value in different subfields.

'''

In [ ]:

# ! pip install langchain-ollama "ollama==0.4.2"  -q

In [ ]:
import re

def extract_json(text):
    """
    Extracts the JSON content from a string, 
    handling cases with or without ```json\n markers.

    Args:
        text: The input string containing the JSON content.

    Returns:
        The extracted JSON string, or None if no match is found or invalid JSON.
    """
    try:
        # First, try to load the entire string as JSON
        json_content = json.loads(text)
        return json.dumps(json_content, indent=2)  # Reformat for consistent output
    except json.JSONDecodeError:
        # If the entire string is not valid JSON, try the previous regex method
        pattern = r"```json\n(.*?)\n```"
        match = re.search(pattern, text, re.DOTALL)
        if match:
            return match.group(1).strip()
        else:
            return None

In [ ]:
# ELastic Query Model

from langchain_ollama import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
import pprint

# LLM
llm = ChatOllama(model="deepseek-r1:1.5b",  temperature=0.8, top_p=.9, tok_k=30, num_ctx=4096)

prompt2 = PromptTemplate(
    template="""A conversation between User and Assistant. The user asks a question, and the Assistant solves it.
    Your task is to interpret user questions about personal profiles and generate the appropriate Elasticsearch query in JSON format.
    The assistant first thinks about the reasoning process in the mind and then provides the user with the answer.
    The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively.

User: {question}.
Assistant: <think>
1. Analyze the question in natural Language that the user provided
2. Analyze the Elastic Schema and the examples provided here \n {prompt} \n
3. Apply relevant rules to build an appropiate Elastic query
4. Ensure that the response is  a json compatible query to be send to Elastic
5. create the query step by step
6. Verify the result
</think>
<answer>
[query  solution will be provided here]
</answer>
    """,
    input_variables=["question", "prompt"],
)

elastic_llm = prompt2 | llm 


# LLama 3.2 3Billions

In [ ]:
question = "All non-Singaporean men over the age of 25 who are software people living in woodlands"

response =elastic_llm.invoke({"question": question, "prompt": prompt})



In [ ]:
response_text = extract_json(response.content)
response_text

In [ ]:
es_query= json.loads(response_text)

In [ ]:
es_query= {
  "query": {
    "bool": {
      "should": [
        { "match": { "nationality": "non-singaporean" } },
        { "match": { "gender": "male" } },
        { "range": { "age": { "gte": 25 } } },
        { 
          "match": { 
            "occupation": { 
              "query": "software developer", 
              "fuzziness": "AUTO" 
            } 
          } 
        }
      ],
      "minimum_should_match": 2
    }
  },
  "multi_match": { 
    "query": "software developer",
    "fields": ["occupation", "job_title", "role"],
    "type": "best_fields"
  }
}

In [ ]:
search_results = es_client.search(index=index, body=es_query)

total_hits = search_results['hits']['total']['value']
print(f"Total matches: {total_hits}")

for hit in search_results['hits']['hits']:
    print(f"Score: {hit['_score']}")
    print(f"Name: {hit['_source']['name']}")
    print(f"Age: {hit['_source']['age']}")
    print(f"Gender: {hit['_source']['gender']}")
    print(f"Citizenship: {hit['_source']['citizenship']}")
    print(f"Occupation: {hit['_source']['occupation']}")
    print(f"Address: {hit['_source']['address']}")
    print("---")

# Gemini

In [ ]:
query="All non-Singaporean men over the age of 25 who are software people living in woodlands" 
response=LLM.generate_non_streaming_response(query, system_prompt=prompt)


In [ ]:
response

In [ ]:
es_query=json.loads(response)
pprint(es_query)

In [ ]:
search_results = es_client.search(index=index, body=es_query)

total_hits = search_results['hits']['total']['value']
print(f"Total matches: {total_hits}")

for hit in search_results['hits']['hits']:
    print(f"Score: {hit['_score']}")
    print(f"Name: {hit['_source']['name']}")
    print(f"Age: {hit['_source']['age']}")
    print(f"Gender: {hit['_source']['gender']}")
    print(f"Citizenship: {hit['_source']['citizenship']}")
    print(f"Occupation: {hit['_source']['occupation']}")
    print(f"Address: {hit['_source']['address']}")
    print("---")

# LLama 3.2 3B

In [ ]:
question = "Women who are not alive currently, who are universal blood donors born in singapore"

response =elastic_llm.invoke({"question": question, "prompt": prompt})

es_query=json.loads(response.content)
pprint(es_query)

In [ ]:
search_results = es_client.search(index=index, body=es_query)

total_hits = search_results['hits']['total']['value']
print(f"Total matches: {total_hits}")

for hit in search_results['hits']['hits']:
    print(f"Score: {hit['_score']}")
    print(f"Name: {hit['_source']['name']}")
    print(f"Blood Type: {hit['_source']['blood_type']}")
    print(f"Gender: {hit['_source']['gender']}")
    print(f"Country of Birth: {hit['_source']['country_of_birth']}")
    print(f"Deceased: {hit['_source']['deceased']}")
    print("---")

In [ ]:
query="Women who are not alive currently, who are universal blood donors born in singapore" 

response=LLM.generate_non_streaming_response(query, system_prompt=prompt)
es_query=json.loads(response)
pprint(es_query)

In [ ]:
search_results = es_client.search(index=index, body=es_query)

total_hits = search_results['hits']['total']['value']
print(f"Total matches: {total_hits}")

for hit in search_results['hits']['hits']:
    print(f"Score: {hit['_score']}")
    print(f"Name: {hit['_source']['name']}")
    print(f"Blood Type: {hit['_source']['blood_type']}")
    print(f"Gender: {hit['_source']['gender']}")
    print(f"Country of Birth: {hit['_source']['country_of_birth']}")
    print(f"Deceased: {hit['_source']['deceased']}")
    print("---")

# LLama 3.2 3B

In [ ]:
question = "People with height equal to 175 centimeters" 

response =elastic_llm.invoke({"question": question, "prompt": prompt})

es_query=json.loads(response.content)
pprint(es_query)

In [ ]:
search_results = es_client.search(index=index, body=es_query)

total_hits = search_results['hits']['total']['value']
print(f"Total matches: {total_hits}")

for hit in search_results['hits']['hits']:
    print(f"Score: {hit['_score']}")
    print(f"Name: {hit['_source']['name']}")
    print(f"languages: {hit['_source']}")
    print("---")

# Gemini 1.5 Pro

In [ ]:
query="People which height is equal to 175 centimeters" 

response=LLM.generate_non_streaming_response(query, system_prompt=prompt)
es_query=json.loads(response)
pprint(es_query)

In [ ]:
search_results = es_client.search(index=index, body=es_query)

total_hits = search_results['hits']['total']['value']
print(f"Total matches: {total_hits}")

for hit in search_results['hits']['hits']:
    print(f"Score: {hit['_score']}")
    print(f"Name: {hit['_source']['name']}")
    print(f"languages: {hit['_source']}")
    print("---")